In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os, sys, gc, random, multiprocessing, glob, time
import html

DATA_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/data/'

In [4]:
!pip install transformers
!pip install swifter
!pip install dgl

In [5]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

from scipy.stats import spearmanr

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,DistilBertTokenizer,DistilBertModel
)

from tqdm import tqdm
print(transformers.__version__)

2.8.0


In [6]:
os.listdir(DATA_DIR)

['train', 'valid', 'multimodal_labels.txt']

In [7]:
from tqdm import tqdm_notebook as tqdm
import pickle
import base64
import swifter
import gensim
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


## Data 読み込み

In [8]:
train = pd.read_csv(DATA_DIR+'train/train.sample.tsv', sep='\t')
print(train.shape)
train.head()

(10000, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,AAAAAAAAQEAAAKlDAABPQwAAb0MAALBBAICmQwAATEM=,AAAAAAAAAAAnNsFAAAAAAOeVGD0AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAAA==,treble popular reed,923271
1,101783080,80,61,1,AACgQAAAoEAAAJZCAABgQg==,AAAAADnKfjyatb8/AAAAALxPBz0MH6E9xwkoPAAAAAAAAA...,HAAAAAAAAAA=,compatible ink cartridges,234031
2,100492797,750,750,1,AAAsQgCAg0MAgCREAADqQw==,AAAAAAAAAABmUlE/AAAAABvc0jwAAAAAAAAAAAAAAAAAAA...,BwAAAAAAAAA=,check new look facial cream,199926
3,101708088,480,640,5,AAB6QwCA0kMAANBDAMAeRAAAfEMAAKBAAIDWQwAAVkMAAH...,AAAAAAAAAAAAAAAAAAAAAAXyfz4AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAABwAAAAAAAAAHAAAAAAAAAAcAA...,royal commemorative stamp,772725
4,100316842,800,800,4,AAAKQwCAgEMAADNEAAAHRAAAIkMAwCZEAACqQwDARkQAAE...,AAAAADSdaTwVTdA8AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAGAAAAAAAAAAYAAAAAAAAAHAAAAAAAAAA=,calendula baby shower gel,362576


In [9]:
valid = pd.read_csv(DATA_DIR+'valid/valid.tsv', sep='\t')
print(valid.shape)
valid.head()

(14720, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103047314,800,800,2,AACoQgAA5EIAAKdDAIDuQwCAq0MAAJBCAMA3RABAHkQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
1,103030678,800,800,9,AIDRQwAAoEAAwEJEAABpQwAAA0MAgMlDAABHRAAAIkQAgL...,AAAAAAAAAACdFBI7AAAAAAAAAABu+j0+AAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAAHQAAAAAAAAAEAA...,breathable and comfortable children's shoes,0
2,103031217,488,750,2,AABGQwAANEIAgNNDAADsQwAAUEIAALVDAADRQwCALEQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
3,103026621,929,800,10,AACtQwCAA0QAgPxDAIAoRAAAekMAALdDAACwQwCA30MAgK...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAdAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAJAA...,breathable and comfortable children's shoes,0
4,103017048,800,800,2,AAA+QwCAi0MAgAtEAMA0RAAAqEMAAHhCAMA1RABADUQ=,AAAAAAAAAADIb2s9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0


In [0]:
def transform_boxes(df_row):
    return np.frombuffer(base64.b64decode(df_row.boxes), dtype=np.float32).reshape(df_row.num_boxes, 4)

def transform_features(df_row):
    return np.frombuffer(base64.b64decode(df_row.features), dtype=np.float32).reshape(df_row.num_boxes, 2048)

def transform_class_labels(df_row):
    return np.frombuffer(base64.b64decode(df_row.class_labels), dtype=np.int64).reshape(df_row.num_boxes)

def transform_df(df):
  df.boxes = df.swifter.apply(transform_boxes, axis=1)
  df.features = df.swifter.apply(transform_features, axis=1)
  df.class_labels = df.swifter.apply(transform_class_labels, axis=1)
  return df

In [11]:
train = transform_df(train)
valid = transform_df(valid)

In [0]:
# boxの面積
train['box_area'] = train.boxes.apply(lambda x: np.array([(xi[2]-xi[0])*(xi[3]-xi[1]) for xi in x]))
valid['box_area'] = valid.boxes.apply(lambda x: np.array([(xi[2]-xi[0])*(xi[3]-xi[1]) for xi in x]))

In [0]:
# train['area_order'] = train.box_area.apply(lambda x: np.array(x).argsort()[::-1])
# valid['area_order'] = valid.box_area.apply(lambda x: np.array(x).argsort()[::-1])

In [14]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_area
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[28, 28]",treble popular reed,923271,"[68952.0, 17108.0]"
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[28],compatible ink cartridges,234031,[3570.0]
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...",[7],check new look facial cream,199926,[126075.0]
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[28, 28, 28, 28, 28]",royal commemorative stamp,772725,"[35524.0, 36993.0, 40936.0, 5472.0, 39078.0]"
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[28, 6, 6, 28]",calendula baby shower gel,362576,"[163574.0, 22784.0, 20988.0, 138230.0]"


In [0]:
class_label_dict = {
    0:'top clothes (coat, jacket, shirt, etc.)',
    1:'skirt & dress',
    2:'bottom clothes (trousers, pants, etc.)',
    3:'luggage, leather goods',
    4:'shoes',
    5:'accessories (jewelry, clothing accessories, belts, hats, scarves, etc.)',
    6:'snacks, nuts, liquor and tea',
    7:'makeup, perfume, beauty tools and essential oils',
    8:'bottle drink',
    9:'furniture',
    10:'stationery',
    11:'household electrical appliances',
    12:'home decoration',
    13:'household fabric',
    14:'kitchenware',
    15:'home / personal cleaning tools',
    16:'storage supplies',
    17:'motorcycle, motorcycle accessories, vehicles, bicycle and riding equipment',
    18:'outdoor product',
    19:'lighting',
    20:'toys',
    21:'underwear',
    22:'digital supplies',
    23:'bed linens',
    24:'baby products',
    25:'personal care',
    26:'sporting goods',
    27:'clothes (accessories, baby clothing, etc.)',
    28:'others',
    29:'human face',
    30:'arm',
    31:'hair',
    32:'hand',
}

In [86]:
l=[1,2,3]
l.replace(class_label_dict)

AttributeError: ignored

In [87]:
train.class_labels = train.class_labels.swifter.apply(lambda x: [class_label_dict[xi] for xi in x])
valid.class_labels = valid.class_labels.swifter.apply(lambda x: [class_label_dict[xi] for xi in x])

In [88]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_area,center,d_mat,center_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[others, others]",treble popular reed,923271,"[68952.0, 17108.0]","[[338.0, 204.0], [94.0, 182.0]]","[[0.0, 1.3707498368934106], [1.370749836893410...","[[0.9760765550239234, 1.0], [0.870813397129186..."
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[others],compatible ink cartridges,234031,[3570.0],"[[70.0, 51.0]]",[[0.0]],"[[0.8360655737704918, 0.875]]"
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...","[makeup, perfume, beauty tools and essential o...",check new look facial cream,199926,[126075.0],"[[615.0, 205.0]]",[[0.0]],"[[0.2733333333333333, 0.82]]"
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[others, others, others, others, others]",royal commemorative stamp,772725,"[35524.0, 36993.0, 40936.0, 5472.0, 39078.0]","[[166.0, 214.0], [177.0, 209.0], [172.0, 238.0...","[[0.0, 41.30209216989538, 25.298157281508917, ...","[[0.334375, 0.3458333333333333], [0.3265625, 0..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[others, snacks, nuts, liquor and tea, snacks,...",calendula baby shower gel,362576,"[163574.0, 22784.0, 20988.0, 138230.0]","[[578.0, 283.0], [178.0, 128.0], [132.0, 159.0...","[[0.0, 1.8648825825884439, 1.7281718745782393,...","[[0.35375, 0.7225], [0.16, 0.2225], [0.19875, ..."


## Graph Transformer
考え方
- ボックスの中心の近さによって、グラフ構造を作成
- 距離のアテンションと、ボックスの大きさのアテンションをつける

In [15]:
import dgl
import dgl.function as fn

Using backend: pytorch


### グラフ構造作成
- 隣接行列作成：
  - boxがN個あるとする
  - 最初は、N*Nの隣接行列で全結合グラフを作成
  - 各box間の距離を計算して、その距離を画像サイズで正規化して、その逆数を隣接行列の値に代入する
  - （ある閾値以下なら、0にする）


- GNNをやるが、そこでbox面積のattentonも入れる？

In [69]:

train.apply(lambda x: np.array([[(xi[3]-xi[1])/x.image_w, (xi[2]-xi[0])/x.image_h] for xi in x.boxes]), axis=1)

0       [[0.9760765550239234, 1.0], [0.870813397129186...
1                           [[0.8360655737704918, 0.875]]
2                            [[0.2733333333333333, 0.82]]
3       [[0.334375, 0.3458333333333333], [0.3265625, 0...
4       [[0.35375, 0.7225], [0.16, 0.2225], [0.19875, ...
                              ...                        
9995    [[0.14875, 0.2675], [0.48625, 0.23625], [0.268...
9996    [[0.49375, 0.4775], [0.14375, 0.1425], [0.1237...
9997    [[0.68375, 0.79375], [0.1625, 0.16625], [0.158...
9998              [[0.35625, 0.2475], [0.56375, 0.40875]]
9999           [[0.9722222222222222, 0.9338624338624338]]
Length: 10000, dtype: object

In [68]:
train.apply(lambda x: np.array([[(xi[3]-xi[1]), (xi[2]-xi[0])] for xi in x.boxes]), axis=1)

0                         [[204.0, 338.0], [182.0, 94.0]]
1                                          [[51.0, 70.0]]
2                                        [[205.0, 615.0]]
3       [[214.0, 166.0], [209.0, 177.0], [238.0, 172.0...
4       [[283.0, 578.0], [128.0, 178.0], [159.0, 132.0...
                              ...                        
9995     [[119.0, 214.0], [389.0, 189.0], [215.0, 171.0]]
9996    [[395.0, 382.0], [115.0, 114.0], [99.0, 98.0],...
9997    [[547.0, 635.0], [130.0, 133.0], [127.0, 125.0...
9998                     [[285.0, 198.0], [451.0, 327.0]]
9999                                     [[980.0, 706.0]]
Length: 10000, dtype: object

In [0]:
# top/left/bottom/right
train['center_normalized'] = train.apply(lambda x: np.array([[(xi[3]-xi[1])/x.image_w, (xi[2]-xi[0])/x.image_h] for xi in x.boxes]), axis=1)

In [0]:
from numba import jit
@jit
def get_dmat(X, Y, dmat):
    n_box = dmat.shape[0]
    if n_box!=1:
        for i in range(n_box):
            for j in range(i+1, n_box):
                d = np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2)
                dmat[i, j] = 1/(d+1e-7)
                dmat[j, i] = 1/(d+1e-7)
                
    return dmat

def get_dmat_row(x):
    n_box = x.num_boxes
    dmat = np.zeros((n_box, n_box))
    return get_dmat(x.center_normalized[:,0], x.center_normalized[:,1], dmat)

In [79]:
train['d_mat'] = train.swifter.apply(lambda x: get_dmat_row(x), axis=1)

In [80]:
train.iloc[3]

product_id                                                   101708088
image_h                                                            480
image_w                                                            640
num_boxes                                                            5
boxes                [[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...
features             [[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....
class_labels                                      [28, 28, 28, 28, 28]
query                                        royal commemorative stamp
query_id                                                        772725
box_area                  [35524.0, 36993.0, 40936.0, 5472.0, 39078.0]
center               [[166.0, 214.0], [177.0, 209.0], [172.0, 238.0...
d_mat                [[0.0, 41.30209216989538, 25.298157281508917, ...
center_normalized    [[0.334375, 0.3458333333333333], [0.3265625, 0...
Name: 3, dtype: object

In [77]:
train.iloc[3].center_normalized

array([[0.334375  , 0.34583333],
       [0.3265625 , 0.36875   ],
       [0.371875  , 0.35833333],
       [0.1125    , 0.15833333],
       [0.365625  , 0.34791667]])

In [81]:
train['d_mat'].head()

0    [[0.0, 1.3707498368934106], [1.370749836893410...
1                                              [[0.0]]
2                                              [[0.0]]
3    [[0.0, 41.30209216989538, 25.298157281508917, ...
4    [[0.0, 1.8648825825884439, 1.7281718745782393,...
Name: d_mat, dtype: object

## DataLoader

In [0]:
class KDDDataset(Dataset):

    def __init__(self, df, train_mode=True, transform=None):
        self.df = df
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.transform = transform
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # query side
        query = row['query']
        inputs_query = self.tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )
        ids = inputs_query["input_ids"]
        token_type_ids = inputs_query["token_type_ids"]
        mask = inputs_query["attention_mask"]

        # image side
        class_labels = row['class_labels']
        inputs_query = self.tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )
        ids = inputs_query["input_ids"]
        token_type_ids = inputs_query["token_type_ids"]
        mask = inputs_query["attention_mask"]


        adj_mat = row['d_mat']
        box_area = row['box_area']
        
        


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'adj_mat': torch.tensor(adj_mat, dtype=torch.float).squeeze(),
                
        }

In [0]:
MAX_LEN = 512

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = BertTokenizer.from_pretrained(BERT_DIR+'/bert-base-uncased')
        #self.tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased/')
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]

        question_title = row.question_title
        question_body = row.question_body
        answer_text = row.answer


        inputs_q = self.tokenizer.encode_plus(
            question_title + " " + question_body,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )

        inputs_a = self.tokenizer.encode_plus(
            answer_text,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )

        ids_q = inputs_q["input_ids"]
        token_type_ids_q = inputs_q["token_type_ids"]
        mask_q = inputs_q["attention_mask"]

        ids_a = inputs_a["input_ids"]
        token_type_ids_a = inputs_a["token_type_ids"]
        mask_a = inputs_a["attention_mask"]
        
        padding_length_q = MAX_LEN - len(ids_q)
        padding_length_a = MAX_LEN - len(ids_a)
        
        ids_q = ids_q + ([0] * padding_length_q)
        mask_q = mask_q + ([0] * padding_length_q)
        token_type_ids_q = token_type_ids_q + ([0] * padding_length_q)

        ids_a = ids_a + ([0] * padding_length_a)
        mask_a = mask_a + ([0] * padding_length_a)
        token_type_ids_a = token_type_ids_a + ([0] * padding_length_a)
        
        if self.labeled:
            labels = self.get_label(row)
            return {
                'ids_q': torch.tensor(ids_q, dtype=torch.long),
                'mask_q': torch.tensor(mask_q, dtype=torch.long),
                'token_type_ids_q': torch.tensor(token_type_ids_q, dtype=torch.long),
                'ids_a': torch.tensor(ids_a, dtype=torch.long),
                'mask_a': torch.tensor(mask_a, dtype=torch.long),
                'token_type_ids_a': torch.tensor(token_type_ids_a, dtype=torch.long),
                'labels': labels, 
            }
        else:
            return {
                'ids_q': torch.tensor(ids_q, dtype=torch.long),
                'mask_q': torch.tensor(mask_q, dtype=torch.long),
                'token_type_ids_q': torch.tensor(token_type_ids_q, dtype=torch.long),
                'ids_a': torch.tensor(ids_a, dtype=torch.long),
                'mask_a': torch.tensor(mask_a, dtype=torch.long),
                'token_type_ids_a': torch.tensor(token_type_ids_a, dtype=torch.long)
            }


    def get_label(self, row):
        return torch.tensor(row[target_columns].values.astype(np.float32))

In [0]:
def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)

## Image Model

## Query Model